# 0. Review
## 0.A Scikit-Learn

Scikit-Learn is a machine learning python package. It allows users to access machine learning algorithms via **object-oriented programming**.

## 0.B Data Set

I will be using a dataset of avocado prices.

We would like to learn prices of avocado given brand, location sold, total demand, etc.

## 0.C Load Data

Now, we load our training and test set. Run the code below to load 

In [ ]:
import pandas as pd

# load explanatory variables
avocado_training_set  = pd.read_csv("datasets/avocado_training_set",index_col=0)
avocado_training_set['year'] = avocado_training_set['year'].astype(str)
avocado_training_set['Month'] = avocado_training_set['Month'].astype(str)

avocado_test_set = pd.read_csv("datasets/avocado_test_set",index_col=0)
avocado_test_set['year'] = avocado_test_set['year'].astype(str)
avocado_test_set['Month'] = avocado_test_set['Month'].astype(str)

# load predictors
prices_training_set = pd.read_csv("datasets/avocado_prices_training_set",index_col=0)
prices_test_set = pd.read_csv("datasets/avocado_prices_test_set",index_col=0)

In the previous section, we hot-one encoded some categorical variables in the training and test set. In this section, we hot-one encode all categorical variables.

In [ ]:
categorical_variables_ = ['Month','region','type','year']

# split all categorical variables for training set
avocado_split_ = pd.get_dummies(data=avocado_training_set[categorical_variables_],
                                     drop_first=True)
avocado_training_set_cleaned = pd.concat([avocado_training_set,avocado_split_], 
                                     axis=1, sort=False)
avocado_training_set_cleaned = avocado_training_set_cleaned.drop(categorical_variables_,axis=1,
                                                         inplace=False)

In [ ]:
categorical_variables_ = ['Month','region','type','year']

# split all categorical variables 
avocado_split_ = pd.get_dummies(data=avocado_test_set[categorical_variables_],
                                     drop_first=True)
avocado_test_set_cleaned = pd.concat([avocado_test_set,avocado_split_], 
                                     axis=1, sort=False)
avocado_test_set_cleaned = avocado_test_set_cleaned.drop(categorical_variables_,axis=1,
                                                         inplace=False)

**In this section, we will learning how to certain the most accurate linear regression model using our training data.**

# 10.A Linear Regression

2. Then, train a **machine learning model** using **labeled data**

    - "Labeled data" has been labeled with the outcome
    - "Machine learning model" learns the relationship between the attributes of the data and its outcome

Linear regression assumes that there is a *linear* relationship between the explanatory variables and the outcome. 

That our case linear regression means that price does up or down, but not both, at a constant rate if an explanatory variable increases.

Thus, linear regression assumes a linear model of price

$$\text{Price} = \beta_0 + \beta_1 \text{Total Volume} + \beta_2 \text{Year} + \beta_3 \text{Month} + \beta_4 \text{Type}.$$

In a linear regression model, we aim learn the coefficients, $\beta_0 ,\beta_1 ,\beta_2 ,\beta_3,\beta_4$ that minimizes the mean squared error between the model and true response variables (prices). 

That is,
$$\min_{\beta_0 ,\beta_1 ,\beta_2 ,\beta_3,\beta_4} \frac{1}{N}\sum_{i=1}^{N}\left(y_i - \beta_0 + \beta_1 \text{Total Volume}_i + \beta_2 \text{Year}_i + \beta_3 \text{Month}_i + \beta_4 \text{Type}_i\right)^2$$

In learning a model, we will be to predict future prices and study how the explanatory variables affect price.
    
## 10.B Check Training Set

Let's check if we loaded the correct dataset.

In [ ]:
#print head of labels_training_set
prices_training_set.head()

In [ ]:
#print head of  DNA_training_set

avocado_training_set_cleaned.head()

In [ ]:
avocado_training_set_cleaned.columns

## 10.C Statistical Analysis

In the avocado dataset, we have many explanatory variables. **How do we pick the explanatory variables that best explain the variation in the prices?**

Interestingly, any random explanatory variable (as long as this variable is not colinear with another explanatory variable) will increase the $R^2$ value. Thus, increasing the accuracy of the dataset. However, this increase in accuracy may be insignificant. **How determine explanatory variables meaningfully reduce the error in the model?**

Typically, one uses Anova and p-values of coefficients to choose to the ```best``` explanatory variable.

*Neither does Scikit-learn allow for Anova to compare linear model nor does Scikit-learn  compute P values of coefficients of a linear model.* 

If you would like to do such statistical analysis, you need to use the ```statsmodels``` package. There maybe a workshop on this package in the summer.

While Scikit cannot do such analysis, I argue that Anova and p-values are not the most effective ways of comparing linear models. 

- Anova depends on the order of term by term comparison
- p-values of coefficients change depending other coefficients used in the model.

## 10.D Overfitting

It is important that we choose the most variables to put in the linear model. It is also important we do *overfit*. Overfitting is learning the training set too closely. 

Any sample population is not representative of the general population. Additionally, most data contain noise. A model that learns the training set to the T can reproduce behaviors that are mostly seen in the sample population.


The test error is a heuristic used to measure fidelity to the general population. The test set is another sample of the general population that the model has not seen before. If the model performs poorly on the test set, then it is likely to perform poorly on the general population. 

# 10.E.1 Model Selection: LASSO

LASSO is a reformulation of the linear regression problem that removes coefficients that do not decrease the training error by a relatively large amount.

Previously, we had the regression problem as: 
$$\min_{\beta_0 ,\beta_1 ,\beta_2 ,\beta_3,\beta_4} \sum_{i=1}^{N}\left(y_i - \left(\beta_0 + \beta_1 \text{Total Volume}_i + \beta_2 \text{Year}_i + \beta_3 \text{Month}_i + \beta_4 \text{Type}_i\right)\right)^2.$$

LASSO reformulates the regression as:

$$\min_{\beta_0 ,\beta_1 ,\beta_2 ,\beta_3,\beta_4} \sum_{i=1}^{N}\left(y_i - \left(\beta_0 + \beta_1 \text{Total Volume}_i + \beta_2 \text{Year}_i + \beta_3 \text{Month}_i + \beta_4 \text{Type}_i\right)\right)^2 + \alpha \sum_{j=1}^{4}|\beta_j|.$$

$\alpha$ weighs eliminating non-zero coefficients against a decrease in training error.

### 10.E.2 Building a linear model with ```Lasso```

#### I. Initialize linear model 

In [ ]:
from sklearn.linear_model import Lasso

#initialize Lasso with alpha=0.01, fit_intercept=True, normalize=False
# store as lasso

#### II. Fitting linear model

In [ ]:
# store avocado_training_set_cleaned[['Total Volume','year_2016','year_2017', 'year_2018']] as
# explanatory variable



# print explanatory varaible


In [ ]:
# fit lasso with avocado_training_set_cleaned and prices_training_set


#### III. Getting parameters

In [ ]:
# print coef_


In [ ]:
# print intercept_


#### IV. Interpretting model 

Our model is now 

$$\text{Price} =2.5112 -0.1014\times\text{Total Demand} + 0.1298 \times \text{year_2017}.$$

Note that our regression model without regularization was

$$\text{Price} = 2.5260 -0.1040 \times \text{Log(Total Demand)}-0.0106 \times \text{year_2016} + 0.1767\times\text{year_2017}+ 0.04261\times\text{year_2018}.$$

We can interpret this reduction in our model as:

"*given $\alpha=0.01$, $\text{year_2016}$ and $\text{year_2018}$ did not provide a large enough reduction in the training error to retain*."

#### V. Plotting model

In [ ]:
# predicted price with .predict(explanatory_data)



In [ ]:
# create index of variables that are 2017 and not 2017 
year_2017 = avocado_training_set_cleaned['year_2017'].astype(bool)
year_not_2017 = (year_2017 == False)

In [ ]:
#initialize plot
import matplotlib.pyplot as plt
%matplotlib inline

plt.figure(figsize=(15,5))

list_years = [year_not_2017,year_2017]
years = ["not 2017","2017"]
colors = ['r','b']
legend_name_scatter= []
legend_name_line= []
for i in range(len(list_years)):
    indices = list_years[i]
    # get data
    training_total_volume_year_ = avocado_training_set_cleaned.loc[indices,"Total Volume"]
    training_price_year_ = prices_training_set[indices]
    predicted_training_prices_year = predict_training_prices_[indices]


    # scatter plot of the demand 
    plt.scatter(training_total_volume_year_, training_price_year_ , s=5,
                alpha=0.3,color=colors[i])
    # plot of linear model 
    plt.plot(training_total_volume_year_, predicted_training_prices_year,c=colors[i])
    legend_name_scatter.extend(['data for ' + years[i]])
    legend_name_line.extend(['predicted model for ' + years[i]])
legend_name_line.extend(legend_name_scatter)
plt.xlabel('log(Total Volume)')
plt.ylabel('Price')
plt.legend('')
plt.legend(legend_name_line)
plt.show()

### 10.E.3 Exercise: Building a linear model with ```Lasso``` with a large $\alpha$

Following the instructions above and the explanatory variables, ```['Total Volume','year_2016','year_2017', 'year_2018']```, build a linear model with Lasso and set ```alpha=1.0```. 

Print the coefficients of the linear model and compare these coefficients to those of the linear model without regularization:

$$\text{Price} = 2.5260 -0.1040 \times \text{Log(Total Demand)}-0.0106 \times \text{2016_year} + 0.1767\times\text{2017_year}+ 0.04261\times\text{2018_year}.$$


In [ ]:
from sklearn.linear_model import Lasso
lasso = Lasso(alpha=1.0, fit_intercept=True, normalize=False)
explanatory_data= avocado_training_set_cleaned[['Total Volume','year_2016',
                                                'year_2017', 'year_2018']]


# enter solution here


### 10.E.4 Exercise: Building a linear model with ```Lasso``` with a small $\alpha$

Following the instructions above and the explanatory variables, ```['Total Volume','year_2016','year_2017', 'year_2018']```, build a linear model with Lasso and set ```alpha= 1e-6```. 

Print the coefficients of the linear model and compare these coefficients to those of the linear model without regularization:

$$\text{Price} = 2.5260 -0.1040 \times \text{Log(Total Demand)}-0.0106 \times \text{2016_year} + 0.1767\times\text{2017_year}+ 0.04261\times\text{2018_year}.$$


In [ ]:
from sklearn.linear_model import Lasso
lasso = Lasso(alpha=1e-6, fit_intercept=True, normalize=False)
explanatory_data= avocado_training_set_cleaned[['Total Volume','year_2016',
                                                'year_2017', 'year_2018']]
# enter solution here



# 10.F Choosing the best $\alpha$

Varying $\alpha$ drastically affect the model we learn. Is there an alpha that most help us find the most effective model?

## 10.F.1 Cross Validation


Ideally, we would like to find an $\alpha$ that minimizes model error in the general population.

<img src="images/10_alpha_model_error.png" alt="Drawing" style="width: 500px;"/>

$\alpha$ is a hyperparameter of machine learning model. A hyperparameter is value that must be set before learning begins.

In machine learning, hyperparameters are learnt using *cross validation*. 

Cross validation is process by which a dataset into smaller datasets and hyperparameters that minimizes the average error on the sub-datasets are learn.


## 10.F.2 Process of Cross Validation

1) Divide the dataset into *folds*. Folds are smaller datasets of a larger dataset.

<img src="images/10_folds.png" alt="Drawing" style="width: 500px;"/>

2) Set the hyperparameter to an intelligent guess.

3) A fold is then removed and, given the hyperparameter above, the model is trained on the remaining folds.

<img src="images/10_cross_validation.png" alt="Drawing" style="width: 500px;"/>

4) Repeat step 3 repeat until each fold as been removed at least once.

5) Vary the hyperparameter.

6) Repeat Steps 3-4.

7) Repeat steps 5-6 until varying hyperparameter always leads to an increase in model error.

**Note: If you divide your dataset into $k$ folds, the cross validation is called *k-fold cross validation*.**

**If you divide your dataset so that each fold contains one data point, the cross validation is called *leave-one-out cross validation*.**


## 10.F.3 Cross Validation in Scikit-Learn

To do a hyperparameter search using the cross validation, we use the ```GridSearchCV``` class. It takes as argument:
- the estimator object you are using
- the hyperparameter points to check
- the scoring method used, for example mean squared error
- the number of folds

#### I. Initialize ```GridSearchCV```

In [ ]:
from sklearn.model_selection import GridSearchCV


# initialize new Lasso class

# set parameter grid, {'alpha' : [1.0,1e-1,1e-2,1e-3,1e-4,1e-5,1e-6,1e-7,1e-8,1e-9]}

# initalize GridSearchCV with lasso, param_grid, scoring='neg_mean_squared_error',cv=10) as lasso_cv



II. Learn best hyperparameter 

In [ ]:
explanatory_data= avocado_training_set_cleaned[['Total Volume','year_2016','year_2017', 'year_2018']]

In [ ]:
# fit lasso_cv with explanatory_data and prices_training_set




In [ ]:
# print lasso_cv.best_params_


# print best_params_['alpha'] and store as best_alpha



#### III. Re-train Linear Model

In [ ]:
from sklearn.linear_model import Lasso

# initalize lasso with alpha=best_alpha, fit_intercept=True, normalize=False)


# fit lasso with explanatory_data and prices_training_set)


# print coef_ and intercept_



Given the choice of variables ```'Total Volume','year_2016', 'year_2017', 'year_2018'```, ```LASSO``` finds the best model to be

$$\text{Price} = 2.6749 -0.122 \times \text{Log(Total Demand)}-0.0027 \times \text{2016_year} + 0.1988\times\text{2017_year}+  0.0481\times\text{2018_year}.$$

This is roughly the same as the linear model without regularization:

$$\text{Price} = 2.5260 -0.1040 \times \text{Log(Total Demand)}-0.0106 \times \text{2016_year} + 0.1767\times\text{2017_year}+ 0.04261\times\text{2018_year}.$$


## 10.F.4 Exercise: Cross Validation with LassoCV.

Using Lasso with cross validation is so common that scikit-learn created a class for it! It is alot simplier than using ```GridSearchCV```. 

With the explanatory variables, ['Total Volume','year_2016','year_2017', 'year_2018'], and response variable, ```prices_training_set```, follow the steps below use ```LassoCV```.


#### I. Initialize

First, you need to initialize ```LassoCV(alphas,cv)``` as ```lasso_cv```. It takes arguments:

- ```alphas``` - a list of alphas
- ```cv``` - the number of folds.

I set the values of arguments below. ```LassoCV``` computes the mean squared error on the remaining folds.

#### II. Fitting 

Next, you need to fit your instance of ```LassoCV``` to the training data. Use ```explanatory_data``` and ```training_response``` to fit your model.

#### III. Get best alpha

After getting the best fitting your instance, ```LassoCV``` stores the best alpha as ```alpha_```. Get the best alpha and store it as ```best_alpha.``` Print the best alpha.

#### IV. Get model parameters

```LassoCV``` also refits ```Lasso``` to the model explanatory data using the best alpha found above. It stores the coefficients, ```coef_``` and ```intercept_```, of Lasso with best alpha. Print these coefficients and interpret associated model.

In [ ]:
#explanatory variables
explanatory_data= avocado_training_set_cleaned[['Total Volume','year_2016',
                                                'year_2017', 'year_2018']]

training_response = prices_training_set.values.ravel()
# reformat prices 
# load LassoCV

from sklearn.linear_model import LassoCV
import numpy as np
# set LassoCV arguments
alpha_list = np.logspace(0,-9,num=10) # list of alphas
k = 10 # number of folds

In [ ]:
# enter solution for I here
# I) initialize LassoCV(alphas=alpha_list,cv=k)



In [ ]:
# enter solution for II here
# II) fit LassoCV with explanatory_data and training_response



In [ ]:
# enter solution for III here
# III) get best alpha, lasso_cv.alpha_



In [ ]:
# enter solution for IV here
# IV) get coefficent, lasso_cv.coef_, and intercept, lasso_cv.intercept_



## 10.F.5 Exercise: Cross Validation with LassoCV.

Repeat the steps in the exercise above to find the best alpha with the explanatory variables, ['Total Volume','type_organic','year_2016','year_2017', 'year_2018'], and response variable, ```prices_training_set```.

Train ```Lasso``` with the best alpha found.

In [ ]:
#explanatory variables
explanatory_data= avocado_training_set_cleaned[['Total Volume','type_organic','year_2016',
                                                'year_2017', 'year_2018']]

training_response = prices_training_set.values.ravel()
# reformat prices 
# load LassoCV

from sklearn.linear_model import LassoCV
# set LassoCV arguments

import numpy as np
alpha_list=np.logspace(0,-9,num=20)
k = 10

In [ ]:
# enter solution here to train LassoCV

# I) initialize LassoCV


# II) fit LassoCV


# III) get best alpha


# IV) get coefficents and intercept

Given the choice of variables ```'Total Volume','type_organic','year_2016', 'year_2017', 'year_2018',``` LASSO finds the best model to be```LASSO``` finds the best model to be

$$\text{Price} = 1.7465 -0.0479 \times \text{Log(Total Demand)} + 0.3203\times\text{type_organic}-0.0272 \times \text{year_2016} +0.1511\times\text{year_2017}$$